# Исследование игровой индустрии 2000-2013 годов

- Автор: Лазарева Елизавета Сергеевна
- Дата: 25.02.2025

### Цели и задачи проекта

<font color='#777778'>Цель проекта - сделать обзор игровых платформ, изучить объёмы продаж игр разных жанров и региональные предпочтения игроков с акцентом на игры в жанре RPG. 

Задачи проекта: 
* Проверка ошибок в данных и их предобработка
* Фильтрация данных
* Категоризаци данных</font>

### Описание данных

<font color='#777778'>Данные /datasets/new_games.csv содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:
* `Name` — название игры.
* `Platform` — название платформы.
* `Year of Release` — год выпуска игры.
* `Genre` — жанр игры.
* `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
* `EU sales` — продажи в Европе (в миллионах проданных копий).
* `JP sales` — продажи в Японии (в миллионах проданных копий).
* `Other sales` — продажи в других странах (в миллионах проданных копий).
* `Critic Score` — оценка критиков (от 0 до 100).
* `User Score` — оценка пользователей (от 0 до 10).
* `Rating` — рейтинг организации ESRB.</font>

### Содержимое проекта



<font color='#777778'>   
    
1. Загрузка данных и знакомство с ними
    
2. Проверка ошибок в данных и их предобработка  
    
3. Фильтрация данных
       
4. Категоризация данных
      
5. Итоговый вывод</font>


---

## 1. Загрузка данных и знакомство с ними¶

Загрузим необходимые библиотеки для анализа данных и данные из датасета `new_games.csv`. Затем выведем основную информацию о данных с помощью метода `info()` и первые строки датафрейма.

In [7]:
#Загрузим библиотеку pandas
import pandas as pd

In [8]:
#Загрузим данные датасета /datasets/new_games.csv
games = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')

In [9]:
#Выведем первые 5 строк датасета и информацию о нем
games.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [10]:
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


Датасет `new_games.csv` содержит 10 столбцов и 16956 строк, в которых предоставлена информация о играх, их продажах, оценках и рейтинге.

Изучим типы данных и их корректность:

- **Числовые значения с плавающей точкой(float64).** Данным типом предсавлено четыре столбца:
    - `Year of Release` содержит информацию о годе выхода игры. Для такой информации рекоммендуется использовать тип `int64`, так как дробной части у года нет, а также понизить разрядность до `int 16`, поскольку значения достаточно малы для этого.
    - `NA sales`и `Other sales` хранят информацию о продажах в миллионах долларов, поэтому такая информация может содержать дробную часть. Для данных стоблцов тип данных `float64` подходит.
    - `Critic Score` содержит целочисленные значения оценки от 0 до 100, поэтому для данного столбца рекомендуется тип `int64`. Также для оптимизации рекомендуется понизить разрядность до `int8`, поскольку максимальная оценка не может быть больше 100.
    
- **Строковые данные(object).** Данным типом представлено шесть столбцов:
    - `Name`, `Platform`, `Genre` и `Rating` содержат строковую информацию(название игры, платформу, жанр и рейтинг организации ESRB), что логично для строковых данных. Здесь тип данных `object` подходит.
    
    - `EU sales` и `JP sales` хранят информацию о продажах в миллионах долларов, поэтому такая информация хранит числа с дробной частью. Для данных стоблцов рекомендуется использовать тип данных `float64`.
    - `User Score` хранит информацию об оценках пользователя от 1 до 10 с дробной частью. Для данного столбца рекомендуется использовать тип данных `float64` и понизить разрядность до `float32`
    
После анализа видно, что четыре из шести столбцов представлены некорректно. В шести столбцах присутствуют пропуски.
Также обратим внимание, что названия столбцов прописаны в неудобном для работы в Python виде и их рекомендуется привести к snace case. В названиях столбцов, связанных с продажами, стоит указать, что речь идет о миллионах долларов.

---

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма


Выведем на экран названия всех столбцов датафрейма и приведем их к стилю `snake case`

In [14]:
#Выведем на экран названия всех столбцов датафрейма
games.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [15]:
#Приведем все столбцы к стилю snake case. Укажем в столбцах, связанных с продажами, что речь идет о миллионах.
games = games.rename(columns = {'Name':'name', 'Platform':'platform', 'Year of Release':'year_of_release', 
                                'Genre':'genre', 'NA sales':'na_sales_mm', 'EU sales':'eu_sales_mm',
                                'JP sales':'jp_sales_mm', 'Other sales':'other_sales_mm', 'Critic Score':'critic_score', 
                                'User Score':'user_score', 'Rating':'rating'})

In [16]:
#Проверим правильность написания столбцов
games.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales_mm',
       'eu_sales_mm', 'jp_sales_mm', 'other_sales_mm', 'critic_score',
       'user_score', 'rating'],
      dtype='object')

### 2.2. Преобразование типов данных

Преобразуем тип данных для `year_of_release`, `critic_score`, `user_score`, `eu_sales_mm` и `jp_sales_mm` в `float`. При обнаружении стороковых значений, заменим из на пропуски.

In [19]:
#Преобразуем тип данных для year_of_release, critic_score в integer,  понизив разрядность и заменив строковые значения на пропуски
for column in ['year_of_release','critic_score', 'user_score', 'eu_sales_mm','jp_sales_mm' ]:
    games[column] = pd.to_numeric(games[column], errors = 'coerce')

In [20]:
#Приведем user_score к числовому типу данных, понизив разрядность и заменив строковые значения на пропуски.
games['user_score'] = pd.to_numeric(games['user_score'], errors = 'coerce', downcast = 'float')

In [21]:
#Преобразуем тип данных для eu_sales_mm и jp_sales_mm в float, понизив разрядность и заменив сттоковые значения на пропуски
for column in ['eu_sales_mm','jp_sales_mm']:
    games[column] = pd.to_numeric(games[column], errors = 'coerce', downcast='float')

In [22]:
#Проверим корректность типов данных
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  float64
 3   genre            16954 non-null  object 
 4   na_sales_mm      16956 non-null  float64
 5   eu_sales_mm      16950 non-null  float32
 6   jp_sales_mm      16952 non-null  float32
 7   other_sales_mm   16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float32
 10  rating           10085 non-null  object 
dtypes: float32(3), float64(4), object(4)
memory usage: 1.2+ MB


### 2.3. Наличие пропусков в данных

Посчитаем количество и процент строк с пропусками в датафрейме.

In [25]:
#Посчитаем количество пропусков в каждом столбце в абсолютном и относительном значениях
games.isna().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales_mm           0
eu_sales_mm           6
jp_sales_mm           4
other_sales_mm        0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [26]:
games.isna().sum() / len(games) * 100

name                0.011795
platform            0.000000
year_of_release     1.621845
genre               0.011795
na_sales_mm         0.000000
eu_sales_mm         0.035386
jp_sales_mm         0.023590
other_sales_mm      0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64

- В столбце `name` 2 пропуска (чуть больше, чем 0,01%). Данные строки рекомендуется удалить, поскольку их количество незначительно и не повлияет на дальнейший анализ, тогда как отсутствие данных может помешать корректному анализу.
- В столбце `year_of_release` 275 пропусков, или 1,6%. Поскольку в дальнешем анализе необходимо отобрать определенный срез данных по годам, рекомендуется удалить данные строки с пропусками.
- В столбце `genre`, также, как и в столбце `name`, 2 пропуска. Можно предположить, что эти пропуски взаимосвязаны(у игры без названия также не указан жанр). Данные строки рекомендуется удалить по той же причине, что и строки в столбце `name`.
- В столбце `critic_score` 8714 пропусков, это больше 50% всех данных. Такое количество пропусков невозможно удалить и некорректно заполнять средним значением, поэтому заменим пропуски на индикатор -1.
- В столбце `user_score` 6804 пропуска, чуть больше 40%. Такое большое количество пропусков также нельзя удалить или заполнить средним значением, поэтому заменим пропуски индикатором -1.
- В столбце `rating` 6871 пропуск, или 40,5%. Данные этого столбца не нужны для дальнейшего анализа, поэтому пропуски нет необходимости обрабаты
- В столбцах `eu_sales_mm` и `jp_sales_mm` 6 и 4 пропуска соответственно, это 0,03% и 0,02%. Эти пропуски заменим на среднее в зависимости от года выхода и платформы.вать.

В данном датафрейме большое количество пропусков. Скорее всего, это связано с тем, что данные получили из открытых источников. Часть данных могла быть не представлена в тех источниках, где они были взяты, часть данных могла быть потеряна по техническим пр Также пропуски могут быть у ранних игр или игр, которые выпущены в назнакомом для критиков и пользователей районе (например, японские и не переведенные на английский язык игры).ичинам.

Удалим пропуски в столбцах `name`, `year_of_release` и `genre`. Заполним значениями-индексами пропуски в столбцах `critic_score` и `user_score`. В столбцах eu_sales_mm и jp_sales_mm заменим пропуски на среднее в зависимости от года выхода и платформы.

In [29]:
#Удалим пропуски в столбцах name, year_of_release и genre
games_cleaned = games.dropna(subset = ['name', 'year_of_release', 'genre'])

In [30]:
#Заполним пропуски в столбцах critic_score и user_score
games_cleaned = games_cleaned.fillna({'critic_score':-1, 'user_score': -1})

In [31]:
#Заменим пропуски в eu_sales_mm и jp_sales_mm на среднее значение в зависимости от названия платформы и года выхода игры
def mean_eu_sales(row):
    if pd.isna(row['eu_sales_mm']):
        group = games_cleaned[(games_cleaned['year_of_release'] == row['year_of_release']) & 
               (games_cleaned['platform'] == row['platform'])]
        return group['eu_sales_mm'].mean()
    else:
        return row['eu_sales_mm']
    
games_cleaned['eu_sales_mm'] = games_cleaned.apply(mean_eu_sales, axis=1)

In [32]:
def mean_jp_sales(row):
    if pd.isna(row['jp_sales_mm']):
        group = games_cleaned[(games_cleaned['year_of_release'] == row['year_of_release']) & 
               (games_cleaned['platform'] == row['platform'])]
        return group['jp_sales_mm'].mean()
    else:
        return row['jp_sales_mm']
    
games_cleaned['jp_sales_mm'] = games_cleaned.apply(mean_jp_sales, axis=1)

Приведем `year_of_release` и `cricic_score` к `integer`, понизив разрядность.

In [34]:
#Преобразуем тип данных для year_of_release, critic_score в integer,  понизив разрядность
for column in ['year_of_release','critic_score']:
    games_cleaned[column] = pd.to_numeric(games_cleaned[column], downcast='integer')

In [35]:
#Проверим отсутствие пропусков и корректность типов данных.
games_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16679 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16679 non-null  object 
 1   platform         16679 non-null  object 
 2   year_of_release  16679 non-null  int16  
 3   genre            16679 non-null  object 
 4   na_sales_mm      16679 non-null  float64
 5   eu_sales_mm      16679 non-null  float64
 6   jp_sales_mm      16679 non-null  float64
 7   other_sales_mm   16679 non-null  float64
 8   critic_score     16679 non-null  int8   
 9   user_score       16679 non-null  float32
 10  rating           9901 non-null   object 
dtypes: float32(1), float64(4), int16(1), int8(1), object(4)
memory usage: 1.3+ MB


### 2.4. Явные и неявные дубликаты в данных



Нормализуем данные в столбцах `genre`, `platform` и `rating`. Проверим столбцы `genre`, `platform`, `rating` и `year_of_release` на неявные дубликаты.

In [38]:
#Приведем столбец genre к нижнему регистру
games_cleaned['genre'] = games_cleaned['genre'].str.lower()

In [39]:
#Проверим столбец genre на наличие неявных дубликатов
unique_genre = games_cleaned['genre'].unique()
unique_genre 

array(['sports', 'platform', 'racing', 'role-playing', 'puzzle', 'misc',
       'shooter', 'simulation', 'action', 'fighting', 'adventure',
       'strategy'], dtype=object)

In [40]:
#Приведем столбец platform к нижнему регистру
games_cleaned['platform'] = games_cleaned['platform'].str.lower()

In [41]:
#Проверим столбец platform на наличие неявных дубликатов
unique_platform = games_cleaned['platform'].unique()
unique_platform

array(['wii', 'nes', 'gb', 'ds', 'x360', 'ps3', 'ps2', 'snes', 'gba',
       'ps4', '3ds', 'n64', 'ps', 'xb', 'pc', '2600', 'psp', 'xone',
       'wiiu', 'gc', 'gen', 'dc', 'psv', 'sat', 'scd', 'ws', 'ng', 'tg16',
       '3do', 'gg', 'pcfx'], dtype=object)

In [42]:
#Приведем столбец rating к верхнему регистру
games_cleaned['rating'] = games_cleaned['rating'].str.upper()
#Проверим столбец rating на наличие неявных дубликатов
unique_rating = games_cleaned['rating'].unique()
unique_rating

array(['E', nan, 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'], dtype=object)

В столбце `rating` заменим значение `K-A` на `E`, так как это сходные категории в разных обозначениях.

In [44]:
#Заменим в столбце rating значение K-A на E
games_cleaned['rating'] = games_cleaned['rating'].replace('K-A', 'E')


 <div class="alert alert-warning">
<h2> Комментарий ревьюера <a class="tocSkip"> </h2>
    
<b>Некоторые замечания и рекомендации💡:</b> 

Здесь хитрая ситуация. У нас есть игры как из Европы, так и из Азии. В основном используется рейтинг ESRB https://ru.wikipedia.org/wiki/Entertainment_Software_Rating_Board 
    
Но некоторых категорий из представленных у нас в нем нет. Фактически, это сходные категории, но в другом обозначении.
    
Например, можно заменить значения 'К-А' на 'Е'.
    
Рекомендую в подобных случаях уточнять информацию в интернете (в реальных проектах на реальной работе также, ну и можно всегда спрашивать у коллег, особенно у тех, кто предоставил данные. Часто такие категории объясняются в паспорте данных. Ну в нашем случае нужно залезть в интернет). 


In [46]:
#Проверим столбец year_of_release на наличие неявных дубликатов
unique_year = games_cleaned['year_of_release'].unique()
unique_year

array([2006, 1985, 2008, 2009, 1996, 1989, 1984, 2005, 1999, 2007, 2010,
       2013, 2004, 1990, 1988, 2002, 2001, 2011, 1998, 2015, 2012, 2014,
       1992, 1997, 1993, 1994, 1982, 2016, 2003, 1986, 2000, 1995, 1991,
       1981, 1987, 1980, 1983], dtype=int16)

Проверим данные датафрейма на наличие явных дубликатов. Найдем сумму дубликатов, выведем дубликаты на экран и удалим все дубликаты.

In [48]:
#Найдем сумму дубликатов в датафрейме
games_cleaned.duplicated(keep='last').sum()

235

In [49]:
#Выведем дубликаты на экран
duplicates = games_cleaned[games_cleaned.duplicated(keep=False)]
duplicates

,name,platform,year_of_release,genre,na_sales_mm,eu_sales_mm,jp_sales_mm,other_sales_mm,critic_score,user_score,rating
267,Batman: Arkham Asylum,ps3,2009,action,2.24,1.31,0.07,0.61,91,8.9,T
268,Batman: Arkham Asylum,ps3,2009,action,2.24,1.31,0.07,0.61,91,8.9,T
367,James Bond 007: Agent Under Fire,ps2,2001,shooter,1.90,1.13,0.10,0.41,72,7.9,T
368,James Bond 007: Agent Under Fire,ps2,2001,shooter,1.90,1.13,0.10,0.41,72,7.9,T
716,God of War: Ascension,ps3,2013,action,1.23,0.63,0.04,0.35,80,7.5,M
...,...,...,...,...,...,...,...,...,...,...,...
16799,Transformers: Prime,wii,2012,action,0.00,0.01,0.00,0.00,-1,-1.0,NaN
16911,Metal Gear Solid V: The Definitive Experience,xone,2016,action,0.01,0.00,0.00,0.00,-1,-1.0,M
16912,Metal Gear Solid V: The Definitive Experience,xone,2016,action,0.01,0.00,0.00,0.00,-1,-1.0,M
16939,The Longest 5 Minutes,psv,2016,action,0.00,0.00,0.01,0.00,-1,-1.0,NaN


In [50]:
#Создадим датафрейм, очищенный от дубликатов
games_no_duplicates = games_cleaned.drop_duplicates()

В результате изучения данных на наличие дубликатов было обнаружено:
* Неявные дубликаты были обнаружены в столбце rating и исправлены.
* Явных дубликатов обнаружено 235, они были удалены.

Посчитаем количество удаленных строк в абсолютном и относительном значении.

In [53]:
#Посчитаем количество удаленных строк в абсолютном значении
deleted_rows = games.shape[0] - games_no_duplicates.shape[0]
deleted_rows

512

In [54]:
#Посчитаем количество удаленных строк в относительном значении
deleted_rows_pers = deleted_rows / games.shape[0] * 100
deleted_rows_pers

3.019580089643784

Для оптимизации работы с данными в датафрейме были сделаны такие изменения типов данных:
- `year_of_release`: тип данных изменён с `float64` на `int16`.
- `critic_score`: тип данных изменён с `float64` на `int8`.
- `eu_sales_mm` и `jp_sales_mm`: тип данных изменён с `object` на `float64`.
- `user_score`: тип данных изменён с `object` на `float32`.

Для нормализации данных `genre` и `platform` были приведены к нижнему регистру, `rating` к верхнему.
Были обнаружены и приведены к общему виду неявные дубликаты в столбце `rating`.
Явных дубликатов обнаружено 235, они были удалены.
Всего при предобработке данных было удалено 512 строк, чуть больше 3% от общего объема датафрейма

## 3. Фильтрация данных

Отберем данные за период с 2000 по 2013 год включительно и сохраним новый срез данных в датафрейме games_actual

In [58]:
#Отберем данные за период с 2000 по 2013 год включительно и сохраним новый срез данных в датафрейме games_actual
games_actual = games_no_duplicates[(games_no_duplicates['year_of_release']) >= 2000 & 
                                   (games_no_duplicates['year_of_release'] <= 2013)]

## 4. Категоризация данных

Разделим все игры по оценкам пользователей и выделим такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала). Заменим значения NaN на неизвестная категория для строк со значением-индикатором.

In [61]:
# Разделим все игры по оценкам пользователей и выделим такие категории: высокая оценка (от 8 до 10 включительно), 
#средняя оценка (от 3 до 8, не включая правую границу интервала) и 
#низкая оценка (от 0 до 3, не включая правую границу интервала).
games_actual['user_score_category'] = pd.cut(games_actual['user_score'], bins = [0, 3, 8, 10], 
                                             labels = ['Низкая оценка', 'Средняя оценка','Высокая оценка'], right = False)

In [62]:
#Заменяем значения NaN на неизвестная категория для строк со значением-индикатором
games_actual['user_score_category'] = games_actual['user_score_category'].cat.add_categories('неизвестная категория').fillna('неизвестная категория')

Разделим все игры по оценкам критиков и выделим такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала). Заменим значения NaN на неизвестная категория для строк со значением-индикатором.

In [64]:
# Разделим все игры по оценкам критиков и выделим такие категории: высокая оценка (от 80 до 100 включительно), 
#средняя оценка (от 30 до 80, не включая правую границу интервала) и 
#низкая оценка (от 0 до 30, не включая правую границу интервала).
games_actual['critic_score_category'] = pd.cut(games_actual['critic_score'], bins = [0, 30, 80, 100], 
                                             labels = ['Низкая оценка', 'Средняя оценка','Высокая оценка'], right = False)

In [65]:
#Заменяем значения NaN на неизвестная категория для строк со значением-индикатором
games_actual['critic_score_category'] = games_actual['critic_score_category'].cat.add_categories('неизвестная категория').fillna('неизвестная категория')

- После категоризации данных проверим результат: сгруппируем данные по выделенным категориям и посчитаем количество игр в каждой категории.

In [67]:
#Сгруппируем данные по оценкам пользователей и посчитаем количество игр в каждой категории
grouped_by_user = games_actual.groupby('user_score_category')['name'].count()
grouped_by_user

C:\Users\stargirl\AppData\Local\Temp\ipykernel_22844\433287156.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_by_user = games_actual.groupby('user_score_category')['name'].count()


user_score_category
Низкая оценка             142
Средняя оценка           4777
Высокая оценка           2544
неизвестная категория    8981
Name: name, dtype: int64

In [68]:
#Сгруппируем данные по оценкам критиков и посчитаем количество игр в каждой категории
grouped_by_critic = games_actual.groupby('critic_score_category')['name'].count()
grouped_by_critic

C:\Users\stargirl\AppData\Local\Temp\ipykernel_22844\1014004771.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped_by_critic = games_actual.groupby('critic_score_category')['name'].count()


critic_score_category
Низкая оценка              59
Средняя оценка           5945
Высокая оценка           1979
неизвестная категория    8461
Name: name, dtype: int64

- Выделим топ-7 платформ по количеству игр, выпущенных за весь актуальный период. Для этого сгруппируем данные по названию платформ и отсортируем значения по убыванию количества игр. После этого выведем первые семь значений.1

In [70]:
#Сгруппируем данные по названиям платформ и и посчитаем количество игр для каждой платформы
grouped_by_platform = games_actual.groupby('platform')['name'].count()

In [71]:
#Отсортируем значения по убыванию количества игр
grouped_by_platform.sort_values( ascending = False)
grouped_by_platform.head(7)

platform
2600     116
3do        3
3ds      512
dc        52
ds      2121
gb        97
gba      811
Name: name, dtype: int64

## 5. Итоговый вывод


Были загружены данные `new_games.csv`. Они содержат 10 столбцов и 16956 строк, в которых предоставлена информация о играх, их продажах, оценках и рейтинге. При первичном знакомстве с денным и их предобработкой были получены такие результаты:
- Названия всех столбцов для удобства были приведены к `snake case`.
- В восьми столбцах (`name`, `year_of_release`, `genre`, `critic_score`, `user_score`, `eu_sales_mm`, `jp_sales_mm`) были обнаружены пропуски. Для столбцов `name`, `year_of_release` и `genre` строки с пропусками были удалены. Для столбцов `critic_score` и `user_score` пропуски были заменены на значения-индексы -1. Для столбцов `eu_sales_mm` и `jp_sales_mm` пропуски были заменены на среднее значение в зависимости от платформы и года выхода игры. 
- Были обработаны и удалены явные дубликаты. Были обработаны и приведены к общему виду неяные дубликаты.
- Всего было удалено 512 строк, это около 3% от общего объема данных.
- Для оптимизации работы с данными в датафрейме были произведены следующие изменения типов данных:
  -  `year_of_release`, `critic_score`: тип данных изменен с `float64` на `int16`.
  -  `user_score`: тип данных изменен с `float64` на `int16`.
  -  `eu_sales_mm` и `jp_sales_mm`:тип данных изменен с `object` на `float32`.
- Для дополнительной работы был выделен срез данных за период с 2000 по 2013 год. Также были созданы столбцы `user_score_category` и `critic_score_category` с категоризацией оценок пользователей и критиков. 
- Были созданы Series:
  -  `grouped_by_user` с количеством игр для каждой категории рейтинга пользователей.
  -  `grouped_by_critic` с количеством игр для каждой категории рейтинга критиков.
  -  `grouped_by_platform` с топ-7 платформ по количеству игр.